# Working with LLMs API

In this notebook, we will illustrate how to send a prompt to LLM APIs for the following providers:
- OpenAI
- Google
- Anthropic

Both Google and Anthropic have their own API SDKs and are compatible with the OpenAI API SDK. Throughout the course, we will use the OpenAI API Python SDK.


General requirements:
- APIs keys
- Python SDKs for the APIs

Let's start by import the os library:

In [1]:
import os

We than define the following prompt:

In [9]:
content_prompt = """
I am working with a dataset that contains information about the Red30 company's product sales online.
I want to create a SQL query that calculates the total sales by state.
"""

Next, we define the number of tokens and temperature:

In [3]:
temperature = 0
max_tokens = 500

# The OpenAI API

In [4]:
from openai import OpenAI
import pandas as pd


In [5]:
openai_api_key = os.environ.get("OPENAI_API_KEY")
client = OpenAI(api_key= openai_api_key)


In [10]:
response_openai = client.chat.completions.create(
    model="gpt-4.1",
    messages=[{"role": "user", "content": content_prompt}],
    temperature=temperature,
    max_tokens= max_tokens,
)


In [7]:
response_openai


ChatCompletion(id='chatcmpl-C8RjQQ8oSp1TERrhaLhmxils5VKKN', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Certainly! To write a SQL query to get the total number of sales by state, you’ll need to know the relevant table and column names. I’ll assume you have a table called `sales` with a column for `state` and a column for each sale (e.g., `sale_id` or `order_id`). If your table or column names are different, just adjust accordingly.\n\nHere’s a general example:\n\n```sql\nSELECT\n  state,\n  COUNT(*) AS total_sales\nFROM\n  sales\nGROUP BY\n  state\nORDER BY\n  total_sales DESC;\n```\n\n**Explanation:**\n- `state`: The column indicating the state where the sale occurred.\n- `COUNT(*)`: Counts the number of sales (rows) for each state.\n- `GROUP BY state`: Groups the results by state.\n- `ORDER BY total_sales DESC`: Orders the results from most to least sales.\n\nIf your sales data is in a different table or the state information i

In [11]:
print(response_openai.choices[0].message.content)


Certainly! To help you write the SQL query, I’ll assume your dataset has at least the following columns:

- `state` (the state where the sale occurred)
- `sales_amount` (the amount of each sale)

If your table is named `sales`, here’s a basic SQL query to calculate total sales by state:

```sql
SELECT
  state,
  SUM(sales_amount) AS total_sales
FROM
  sales
GROUP BY
  state
ORDER BY
  total_sales DESC;
```

**Explanation:**
- `SUM(sales_amount)` calculates the total sales for each state.
- `GROUP BY state` groups the results by state.
- `ORDER BY total_sales DESC` sorts the results from highest to lowest sales.

If your table or column names are different, just adjust them accordingly. If you need to join with another table (for example, if state info is in a `customers` table), let me know and I can help you modify the query!


# Google Gemini API


In [12]:
from google import genai
from google.genai import types


In [13]:
gemini_api_key = os.environ.get("GEMINI_API_KEY")
gemini_client = genai.Client(api_key=gemini_api_key)

In [14]:
response_gemini = gemini_client.models.generate_content(
    model="gemini-2.0-flash",
    contents=content_prompt,
    config=types.GenerateContentConfig(
        max_output_tokens=max_tokens, temperature=temperature
    ),
)


In [15]:
print(response_gemini.candidates[0].content.parts[0].text)


```sql
SELECT
    customer_state,
    SUM(payment_value) AS total_sales
FROM
    orders o  -- Assuming you have an 'orders' table
JOIN
    customers c ON o.customer_id = c.customer_id  -- Assuming you have a 'customers' table and a customer_id to link them
JOIN
    order_payments op ON o.order_id = op.order_id  -- Assuming you have an 'order_payments' table and an order_id to link them
GROUP BY
    customer_state
ORDER BY
    total_sales DESC;  -- Optional: Order the results by total sales in descending order
```

**Explanation:**

1. **`SELECT customer_state, SUM(payment_value) AS total_sales`**: This selects the customer's state and calculates the sum of the `payment_value` for each state.  `SUM(payment_value)` calculates the total sales amount.  `AS total_sales` gives the calculated sum a more descriptive name.

2. **`FROM orders o JOIN customers c ON o.customer_id = c.customer_id JOIN order_payments op ON o.order_id = op.order_id`**: This specifies the tables involved and how they 

## Using the OpenAI API SDK

Let's now illustrate the use the OpenAI API SDK to call the Google Gemini API. You can find more details on the [doc](https://ai.google.dev/gemini-api/docs/openai).

In [16]:
gemini_client2 = OpenAI(
    api_key= gemini_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)


In [17]:
response_gemini2 = gemini_client2.chat.completions.create(
    model="gemini-2.0-flash",
    messages=[{"role": "user", "content": content_prompt}],
    temperature=temperature,
    max_tokens=max_tokens,
)


In [18]:
print(response_gemini2.choices[0].message.content)


```sql
SELECT
    customer_state,
    SUM(payment_value) AS total_sales
FROM
    orders o  -- Assuming you have an 'orders' table
JOIN
    customers c ON o.customer_id = c.customer_id  -- Assuming you have a 'customers' table and a customer_id to link them
JOIN
    order_payments op ON o.order_id = op.order_id  -- Assuming you have an 'order_payments' table and an order_id to link them
GROUP BY
    customer_state
ORDER BY
    total_sales DESC;  -- Optional: Order the results by total sales in descending order
```

**Explanation:**

1. **`SELECT customer_state, SUM(payment_value) AS total_sales`**: This selects the customer's state and calculates the sum of the `payment_value` for each state.  `SUM(payment_value)` calculates the total sales amount.  `AS total_sales` gives the calculated sum a more descriptive name.

2. **`FROM orders o JOIN customers c ON o.customer_id = c.customer_id JOIN order_payments op ON o.order_id = op.order_id`**: This specifies the tables involved and how they 

# Anthropic Claude API

In [19]:
import anthropic


In [20]:
claude_api_key = os.environ.get("CLAUDE_API_KEY")
claude_client = anthropic.Anthropic(api_key=claude_api_key)



In [21]:

response_claude = claude_client.messages.create(
    model="claude-sonnet-4-20250514",
    max_tokens=1024,
    messages=[{"role": "user", "content": content_prompt}],
)


In [22]:
print(response_claude)


Message(id='msg_0113ZAXeTnpSWd293aHs93eh', content=[TextBlock(citations=None, text="I'd be happy to help you create a SQL query to calculate total sales by state for Red30 company's product sales data. Here's a basic query structure:\n\n```sql\nSELECT \n    state,\n    SUM(sales_amount) AS total_sales\nFROM your_table_name\nGROUP BY state\nORDER BY total_sales DESC;\n```\n\nHowever, to provide you with a more accurate query, I'd need to know:\n\n1. **Table name** - What is your table called?\n2. **Column names** - What are the exact names of your columns for:\n   - State information\n   - Sales amount/revenue\n3. **Data structure** - Are there any other relevant columns (like date, product type, etc.)?\n\nHere are a few variations depending on common column naming conventions:\n\n**If your columns are named differently:**\n```sql\n-- If state column is named 'customer_state' and sales is 'revenue'\nSELECT \n    customer_state AS state,\n    SUM(revenue) AS total_sales\nFROM red30_sales

In [23]:
print(response_claude.content[0].text)

I'd be happy to help you create a SQL query to calculate total sales by state for Red30 company's product sales data. Here's a basic query structure:

```sql
SELECT 
    state,
    SUM(sales_amount) AS total_sales
FROM your_table_name
GROUP BY state
ORDER BY total_sales DESC;
```

However, to provide you with a more accurate query, I'd need to know:

1. **Table name** - What is your table called?
2. **Column names** - What are the exact names of your columns for:
   - State information
   - Sales amount/revenue
3. **Data structure** - Are there any other relevant columns (like date, product type, etc.)?

Here are a few variations depending on common column naming conventions:

**If your columns are named differently:**
```sql
-- If state column is named 'customer_state' and sales is 'revenue'
SELECT 
    customer_state AS state,
    SUM(revenue) AS total_sales
FROM red30_sales
GROUP BY customer_state
ORDER BY total_sales DESC;
```

**If you want to include additional details:**
```sql


## Using the OpenAI API SDK

Let's now illustrate the use the OpenAI API SDK to call the Google Gemini API. You can find more details on the [doc](https://docs.anthropic.com/en/api/openai-sdk).

In [24]:
claude_client2 = OpenAI(
    api_key=claude_api_key,
    base_url="https://api.anthropic.com/v1/",
)


In [25]:
response_claude2 = claude_client2.chat.completions.create(
    model="claude-opus-4-1-20250805",
    messages=[{"role": "user", "content": content_prompt}],
    temperature=temperature,
    max_tokens=max_tokens,
)


In [26]:
print(response_claude2.choices[0].message.content)


I'll help you create a SQL query to calculate total sales by state for Red30's online product sales. Since I don't have the exact schema of your dataset, I'll provide a few variations based on common table structures.

## Basic Query (Most Common Structure)

```sql
SELECT 
    state,
    SUM(sales_amount) AS total_sales
FROM 
    sales_table
GROUP BY 
    state
ORDER BY 
    total_sales DESC;
```

## If you have separate order and order_items tables:

```sql
SELECT 
    c.state,
    SUM(oi.quantity * oi.unit_price) AS total_sales
FROM 
    orders o
    JOIN customers c ON o.customer_id = c.customer_id
    JOIN order_items oi ON o.order_id = oi.order_id
GROUP BY 
    c.state
ORDER BY 
    total_sales DESC;
```

## With additional useful information:

```sql
SELECT 
    state,
    COUNT(DISTINCT order_id) AS number_of_orders,
    COUNT(DISTINCT customer_id) AS unique_customers,
    SUM(sales_amount) AS total_sales,
    ROUND(AVG(sales_amount), 2) AS avg_sale_amount
FROM 
    sales_table


# Resources

- OpenAI API documentation
- Gemini API documentation
- Claude API documentation